# Julia-Notebook zur Veranschaulichung der Pol-Nullstellen-Methode

## Initialisierung

In [ ]:
using Interact, Plots, DSP, LaTeXStrings

## Bandpass

Parameter:
* Abtastfrequenz $f_s$
* Bandbreite $B$
* Mittenfrequenz $f_0$
* ein gegebener Wert von $H^\prime(f)=H\left(e^{j2\pi f/ f_s}\right)$, z.B., $H^\prime(f_0)=1$

Nullstellen: $z_{0,1}=1$ und $z_{0,2}=-1$

Polstellen: $z_{\infty,1/2}=r e^{\pm j 2 \pi \frac{f_0}{f_s}}$ mit $r=1-\frac{B}{f_s}\pi$

Funktion zur Ermittlung der Filterkoeffzienten (Annahme: $H^\prime(f_0)=1$):

In [ ]:
function pole_zero_method_BP(f_0_norm, B_norm)
    # Nullstellen
    z = [1, -1];

    # Pole
    r = 1 - B_norm*pi;
    z_0 = exp(im*2*pi*f_0_norm);
    p = [r*z_0, r*conj(z_0)];

    # Berechnung des Koeffizienten b_0 (gain k)
    k = (z_0-p[1])*(z_0-p[2])/(z_0-z[1])/(z_0-z[2]);

    # Filterkoeffizienten
    return ZeroPoleGain(z, p, k);
    
end;

Resultierender Frequenzgang:

In [ ]:
@manipulate for f_0_norm in 0:.01:.5, B_norm in 0.01:.01:.5
    f_norm = 0:.001:.5;
    H = freqz(pole_zero_method_BP(f_0_norm, B_norm), f_norm*2*pi);
    plot(f_norm, 20*log10.(abs.(H)), label=false, lw=2, linecolor = :blue);
    vline!([f_0_norm-B_norm/2], label = L"f = f_0-B/2", linecolor = :green);
    vline!([f_0_norm+B_norm/2], label = L"f = f_0+B/2", linecolor = :green);
    xlims!((0,.5));
    ylims!((-50,5));
    xlabel!(L"f/f_s");
    ylabel!(L"20 \log_{10}H^\prime(f)");
  end

## Bandsperre

Parameter:
* Abtastfrequenz $f_s$
* Bandbreite $B$
* Mittenfrequenz $f_0$
* ein gegebener Wert von $H^\prime(f)=H\left(e^{j2\pi f/ f_s}\right)$, z.B., $H^\prime(0)=1$

Nullstellen: $z_{0,1/2}=e^{\pm j 2 \pi \frac{f_0}{f_s}}$

Polstellen: $z_{\infty,1/2}=r e^{\pm j 2 \pi \frac{f_0}{f_s}}$ mit $r=1-\frac{B}{f_s}\pi$

Funktion zur Ermittlung der Filterkoeffzienten (Annahme: $H^\prime(0)=1$):

In [ ]:
function pole_zero_method_BS(f_0_norm, B_norm)
    # Nullstellen
    z_0 = exp(im*2*pi*f_0_norm);
    z = [z_0, conj(z_0)];

    # Pole
    r = 1 - B_norm*pi;
    p = [r*z_0, r*conj(z_0)];

    # Berechnung des Koeffizienten b_0 (gain k)
    k = (1-p[1])*(1-p[2])/(1-z[1])/(1-z[2]);

    # Filterkoeffizienten
    return ZeroPoleGain(z, p, k);
    
end;

Resultierender Frequenzgang:

In [ ]:
@manipulate for f_0_norm in 0:.01:.5, B_norm in 0.01:.01:.5
    f_norm = 0:.001:.5;
    H = freqz(pole_zero_method_BS(f_0_norm, B_norm), f_norm*2*pi);
    plot(f_norm, 20*log10.(abs.(H)), label=false, lw=2, linecolor = :blue);
    vline!([f_0_norm-B_norm/2], label = L"f = f_0-B/2", linecolor = :green);
    vline!([f_0_norm+B_norm/2], label = L"f = f_0+B/2", linecolor = :green);
    xlims!((0,.5));
    ylims!((-50,5));
    xlabel!(L"f/f_s");
    ylabel!(L"20 \log_{10}H^\prime(f)");
  end